# follow top-to-bottom cell execution.
1. import packages
2. execute utils 
2. there are 4 domains data used here. choose any one from next 4-cells & execute.(Toxic/custom/ice-xd(testAPSSDC)/snips)
3. execute training cell
4. in prediction cell, validation data for each domains data are commented by triple quotes. based on training, choose one.

# pros :
1. best when query variation limited. search always narrows down to specific domain. example : chatbot used in retail, or other domain.
2. performance faster
3. memory consumption less

# cons :
1. can't handle new words like synonym & other form of words.
2. as it works on TF/IDF , it'll give more confidence to label which has more TF/TDF.
3.

In [1]:
from __future__ import division
import re
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV

# utils

In [2]:
import os
import csv
import nltk
import json
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings

def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

# kaggle competition : Toxic Comment Classification Challenge
https://www.kaggle.com/rhodiumbeng/classifying-multi-label-comments-0-9741-lb/data  -- get data into data directory or change below path as per you convenience.

In [3]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
test_labels_df = pd.read_csv('data/test_labels.csv')

#print train_df.sample(5)
#print test_df.sample(5)
test_labels_df = test_labels_df[test_labels_df.toxic != -1]
#print len(test_df)
test_df = test_df[test_df['id'].isin(test_labels_df.id)]
#print len(test_df), len(test_labels_df)
#print test_labels_df.sample(5)

cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

# special case
for col in cols_target:
    train_df[col] = train_df[col].map(lambda com : col if com==1 else 'NA')
    test_labels_df[col] = test_labels_df[col].map(lambda com : col if com==1 else 'NA')

print "train sample"
print train_df.sample(5)

train_labels_df = train_df[cols_target]
# Let's look at the character length for the rows in the training data and record these
#train_df['char_length'] = train_df['comment_text'].apply(lambda x: len(str(x)))
#test_df['char_length'] = test_df['comment_text'].apply(lambda x: len(str(x)))

# explicit check
xx = train_df[(train_df['toxic']=='toxic') & (train_df['obscene']=='obscene')].head(5)
#print xx
#print list(xx['id'])
#print list(xx['comment_text'])

# clean the comment_text in train_df [Thanks to Pulkit Jha for the useful pointer.]
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
train_df['comment_text'] = train_df['comment_text'].str.lower()
train_df['comment_text'] = train_df['comment_text'].apply(cleanHtml)
train_df['comment_text'] = train_df['comment_text'].apply(cleanPunc)
train_df['comment_text'] = train_df['comment_text'].apply(keepAlpha)
train_df['comment_text'] = train_df['comment_text'].apply(removeStopWords)
train_df['comment_text'] = train_df['comment_text'].apply(stemming)

# clean the comment_text in test_df [Thanks, Pulkit Jha.]
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].str.lower()
test_df['comment_text'] = test_df['comment_text'].apply(cleanHtml)
test_df['comment_text'] = test_df['comment_text'].apply(cleanPunc)
test_df['comment_text'] = test_df['comment_text'].apply(keepAlpha)
test_df['comment_text'] = test_df['comment_text'].apply(removeStopWords)
test_df['comment_text'] = test_df['comment_text'].apply(stemming)
#train_df = train_df.drop('char_length',axis=1)

train_X = train_df.comment_text
test_X = test_df.comment_text

target_names = cols_target #target_names
X_train = train_X.values #X_train
y_train_text = [list(set(i).difference(set(["NA"]))) for i in train_labels_df.values.tolist()] #y_train_text
X_test = test_X.values #X_test
print target_names
print X_train[:5]
print y_train_text[:5]
print X_test[:5]

train sample
                      id                                       comment_text  \
105173  32a5fb2f318ea716  "\n\n Suntag's RfA \n\nCould you please elabor...   
141992  f78f2377112245fe  Um.... yeah, I know it is.  Go fuck yourself, ...   
106660  3a7a5dedbb9dc5f6  Chasing each other \n\nI guess you're doing th...   
46      001c419c445b5a59  You had a point, and it's now ammended with ap...   
147402  3bddfc9d94e0b68f  There is no longer a picture because the wikin...   

        toxic  severe_toxic  obscene threat insult identity_hate  
105173     NA            NA       NA     NA     NA            NA  
141992  toxic  severe_toxic  obscene     NA     NA            NA  
106660     NA            NA       NA     NA     NA            NA  
46         NA            NA       NA     NA     NA            NA  
147402     NA            NA       NA     NA     NA            NA  
['obscene', 'insult', 'toxic', 'severe_toxic', 'identity_hate', 'threat']
[u'explan edit made usernam hardcor 

# custom multi-intent datasets

In [45]:
X_train = np.array(["Hi","Hi what's up?","Hello","new york is a hell of a town",
                    "new york was originally dutch",
                    "the big apple is great",
                    "new york is also called the big apple",
                    "nyc is nice",
                    "people abbreviate new york city as nyc",
                    "the capital of great britain is london",
                    "london is in the uk",
                    "london is in england",
                    "london is in great britain",
                    "it rains a lot in london",
                    "london hosts the british museum",
                    "new york is great and so is london",
                    "i like london better than new york",
                    "Hi. london is in the uk",
                    "hello. nyc is nice","Hi bot.","Hi there.","Hello there.","Hey what's up?",
                    "Hey."])
y_train_text = [["greet"],["greet"],["greet"],["new york"],["new york"],["new york"],["new york"],["new york"],
                ["new york"],["london"],["london"],["london"],["london"],
                ["london"],["london"],["new york","london"],["new york","london"],
                ["greet","london"],["greet","new york"],["greet"],["greet"],
               ["greet"],["greet"],["greet"]]

X_test = np.array(['nice day in nyc',
                   'welcome to london',
                   'london is rainy',
                   'is england a london',
                   'it is raining in new york and the big apple',
                   'it is raining in london and nyc',
                   'hello welcome to new york. enjoy it here and london too'
                  'hello there. have you been to london'])
target_names = ['New York', 'London']
y_test = []

# SNIPS intent classification dataset

### https://github.com/snipsco/nlu-benchmark     -- clone data into snips_data directory or change below path as per your convenience.

In [101]:
snips_text_train = []
snips_labels_train = []
snips_text_val = []
snips_labels_val = []

take_n_samples_per_class = 100

import json
BR_train = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/BookRestaurant/train_BookRestaurant_full.json"
BR_val = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/BookRestaurant/validate_BookRestaurant.json"
with open(BR_train) as f:
    BR_train_data = json.load(f)
with open(BR_val) as f:
    BR_val_data = json.load(f)

text_list_BR = []
text_entity_BR = []
label_list_BR = []
text_list_BR_val = []
text_entity_BR_val = []
label_list_BR_val = []

for each_data in BR_train_data['BookRestaurant'][:take_n_samples_per_class]:
    text_list_BR.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_BR.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_BR.append(["BookRestaurant"])
for each_data in BR_val_data['BookRestaurant'][:500]:
    text_list_BR_val.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_BR_val.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_BR_val.append(["BookRestaurant"])

snips_text_train.extend(text_list_BR)
snips_labels_train.extend(label_list_BR)
snips_text_val.extend(text_list_BR_val)
snips_labels_val.extend(label_list_BR_val)

GW_train = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/GetWeather/train_GetWeather_full.json"
GW_val = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/GetWeather/validate_GetWeather.json"
with open(GW_train) as f:
    GW_train_data = json.load(f)
with open(GW_val) as f:
    GW_val_data = json.load(f)

text_list_GW = []
text_entity_GW = []
label_list_GW = []
text_list_GW_val = []
text_entity_GW_val = []
label_list_GW_val = []
for each_data in GW_train_data['GetWeather'][:take_n_samples_per_class]:
    text_list_GW.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_GW.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_GW.append(["GetWeather"])
for each_data in GW_val_data['GetWeather'][:500]:
    text_list_GW_val.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_GW_val.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_GW_val.append(["GetWeather"])

snips_text_train.extend(text_list_GW)
snips_labels_train.extend(label_list_GW)
snips_text_val.extend(text_list_GW_val)
snips_labels_val.extend(label_list_GW_val)

PM_train = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/PlayMusic/train_PlayMusic_full.json"
PM_val = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/PlayMusic/validate_PlayMusic.json"
with open(PM_train) as f:
    PM_train_data = json.load(f)
with open(PM_val) as f:
    PM_val_data = json.load(f)

text_list_PM = []
text_entity_PM = []
label_list_PM = []
text_list_PM_val = []
text_entity_PM_val = []
label_list_PM_val = []
for each_data in PM_train_data['PlayMusic'][:take_n_samples_per_class]:
    text_list_PM.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_PM.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_PM.append(["PlayMusic"])
for each_data in PM_val_data['PlayMusic'][:500]:
    text_list_PM_val.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_PM_val.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_PM_val.append(["PlayMusic"])

snips_text_train.extend(text_list_PM)
snips_labels_train.extend(label_list_PM)
snips_text_val.extend(text_list_PM_val)
snips_labels_val.extend(label_list_PM_val)

RB_train = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/RateBook/train_RateBook_full.json"
RB_val = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/RateBook/validate_RateBook.json"
with open(RB_train) as f:
    RB_train_data = json.load(f)
with open(RB_val) as f:
    RB_val_data = json.load(f)

text_list_RB = []
text_entity_RB = []
label_list_RB = []
text_list_RB_val = []
text_entity_RB_val = []
label_list_RB_val = []
for each_data in RB_train_data['RateBook'][:take_n_samples_per_class]:
    text_list_RB.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_RB.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_RB.append(["RateBook"])
for each_data in RB_val_data['RateBook'][:500]:
    text_list_RB_val.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_RB_val.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_RB_val.append(["RateBook"])

snips_text_train.extend(text_list_RB)
snips_labels_train.extend(label_list_RB)
snips_text_val.extend(text_list_RB_val)
snips_labels_val.extend(label_list_RB_val)

ATP_train = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/AddToPlaylist/train_AddToPlaylist_full.json"
ATP_val = "snips_data/nlu-benchmark/2017-06-custom-intent-engines/AddToPlaylist/validate_AddToPlaylist.json"
with open(ATP_train) as f:
    ATP_train_data = json.load(f)
with open(ATP_val) as f:
    ATP_val_data = json.load(f)

text_list_ATP = []
text_entity_ATP = []
label_list_ATP = []
text_list_ATP_val = []
text_entity_ATP_val = []
label_list_ATP_val = []
for each_data in ATP_train_data['AddToPlaylist'][:take_n_samples_per_class]:
    text_list_ATP.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_ATP.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_ATP.append(["AddToPlaylist"])
for each_data in ATP_val_data['AddToPlaylist'][:500]:
    text_list_ATP_val.append(" ".join([i.get("text","") for i in each_data['data']]).replace("  "," "))
    text_entity_ATP_val.append(" ".join([i.get("entity","") if i.get("entity",None) !=None else i.get("text","") for i in each_data['data']]).replace("  "," "))
    label_list_ATP_val.append(["AddToPlaylist"])

snips_text_train.extend(text_list_ATP)
snips_labels_train.extend(label_list_ATP)
snips_text_val.extend(text_list_ATP_val)
snips_labels_val.extend(label_list_ATP_val)

print "Train"
print snips_text_train[:5],snips_text_train[-5:]
print snips_labels_train[:5],snips_labels_train[-5:]
print len(snips_text_train),len(snips_labels_train)

print "Test"
print snips_text_val[:5],snips_text_val[-5:]
print snips_labels_val[:5],snips_labels_val[-5:]
print len(snips_text_val),len(snips_labels_val)

X_train = np.array(snips_text_train)
y_train_text = snips_labels_train
target_names = ["BookRestaurant","GetWeather","PlayMusic","RateBook","AddToPlaylist"]
X_test = np.array(snips_text_val)
y_org = snips_labels_val

Train
[u'book The Middle East  restaurant in IN for noon', u'Book a table at T-Rex  distant from Halsey St .', u"I'd like to eat at a taverna that serves chili con carne for a party of 10 ", u'I have a party of four in Japan and need a reservation at Rimsky-Korsakoffee House on Aug. the 3rd .', u'Please make a restaurant reservation for somewhere in Mondovi , Connecticut .'] [u'Add song to my  Club Hits', u'add this tune to my playlist titled uncharted 4 nathan drake', u'Add 100% te ljubam to the Hit Remix playlist', u'Put this album on totally alternative', u"put this album into becky's  infinite indie folk playlist"]
[['BookRestaurant'], ['BookRestaurant'], ['BookRestaurant'], ['BookRestaurant'], ['BookRestaurant']] [['AddToPlaylist'], ['AddToPlaylist'], ['AddToPlaylist'], ['AddToPlaylist'], ['AddToPlaylist']]
500 500
Test
[u'Book a reservation for my babies and I', u'Book a reservation for a restaurant  not far from MA', u'I would like to book a restaurant in Tanzania that is within

# Training OneVsRest

In [8]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(y_train_text)
print mlb.classes_

rf = RandomForestClassifier(n_estimators=500, criterion='gini',oob_score=True, class_weight='balanced',n_jobs=-1, verbose=1)

#'''
# Set the parameters by cross-validation
parameters = [{'kernel': ['rbf'],
               'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
                'C': [1, 10, 100, 1000]},
              {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

# this is for SGDclassifier
'''
parameters = {
    'penalty': ('l2','l1','elasticnet'),
    'loss': ('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron','squared_loss', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive'),
    'max_iter': (1000,),
    'alpha': (0.00001, 0.000001),
}
'''
#print("# Tuning hyper-parameters")
#print()

#clf = GridSearchCV(svm.SVC(decision_function_shape='ovr'), parameters, cv=5)
#clf.fit(X_train, y_train)
#'''

classifier = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)),#ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1
    ('tfidf', TfidfTransformer()),
    #('clf', OneVsRestClassifier(LinearSVC()))]) # good at ngrams
    #('clf', OneVsRestClassifier(CalibratedClassifierCV(SGDClassifier())))])
    #('clf', OneVsRestClassifier(CalibratedClassifierCV(GridSearchCV(SGDClassifier(), parameters, cv=5))))])
    #('clf', OneVsRestClassifier(CalibratedClassifierCV(svm.SVC(decision_function_shape='ovr'))))])
    #('clf', OneVsRestClassifier(CalibratedClassifierCV(GridSearchCV(svm.SVC(decision_function_shape='ovr'), parameters, cv=5))))]) # good at ngrams
    #('clf', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC())))]) # good at ngrams
    #('clf', OneVsRestClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True, intercept_scaling=1, loss='squared_hinge', max_iter=1000,multi_class='ovr', penalty='l2')))])
    #('clf', OneVsRestClassifier(LogisticRegression(C=1.,solver='sag'), n_jobs=1))]) #C=1., solver='lbfgs'/'sag'
    #('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None)))]) # bad at ngrams
    ('clf', OneVsRestClassifier(rf))])

classifier.fit(X_train, Y)
#classifier.get_params()

[u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow']


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
    ..., oob_score=True, random_state=None,
            verbose=1, warm_start=False),
          n_jobs=1))])

# Training OneVsOne

In [12]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(y_train_text)
print mlb.classes_

rf = RandomForestClassifier(n_estimators=500, criterion='gini',oob_score=True, class_weight='balanced',n_jobs=-1, verbose=1)

# Set the parameters by cross-validation
parameters = [{'kernel': ['rbf'],
               'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
                'C': [1, 10, 100, 1000]},
              {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

# this is for SGDclassifier
'''
parameters = {
    'penalty': ('l2','l1','elasticnet'),
    'loss': ('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron','squared_loss', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive'),
    'max_iter': (1000,),
    'alpha': (0.00001, 0.000001),
}
'''

classifier = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)),#ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1
    ('tfidf', TfidfTransformer()),
    #('clf', LinearSVC())]) # good at ngrams
    #('clf', SGDClassifier())])
    #('clf', GridSearchCV(SGDClassifier(), parameters, cv=5))])
    #('clf', svm.SVC(decision_function_shape='ovr'))])
    #('clf', GridSearchCV(svm.SVC(decision_function_shape='ovr'), parameters, cv=5))]) # good at ngrams
    #('clf', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True, intercept_scaling=1, loss='squared_hinge', max_iter=1000,multi_class='ovr', penalty='l2'))])
    #('clf', LogisticRegression(C=1.,solver='sag'), n_jobs=1)]) #C=1., solver='lbfgs'/'sag'
    #('clf', MultinomialNB(fit_prior=True, class_prior=None))]) # bad at ngrams
    ('clf',rf)])

classifier.fit(X_train, Y)
#classifier.get_params()

[u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow']


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.7s finished


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
    ...mators=500, n_jobs=-1, oob_score=True, random_state=None,
            verbose=1, warm_start=False))])

In [ ]:
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("\n")

# Prediction OneVsRest

In [9]:
# custom data
'''
predicted = classifier.predict(X_test)
print predicted
#print classifier.predict_proba(X_test)
all_labels = mlb.inverse_transform(predicted)

for item, labels in zip(X_test, all_labels):
    print('{0} => {1}'.format(item, ', '.join(labels)))
'''

# toxic data
'''
test_label_toxic = [tuple(j for j in i if j!="NA") for i in test_labels_df[cols_target].values.tolist()]
pred_true = []
for item, org_labels, pred_labels in zip(X_test, test_label_toxic, all_labels):
    pred_true.append(all(True if i in org_labels else False for i in pred_labels))
    #print('{0} => {1} => {2}'.format(item, ', '.join(org_labels), ', '.join(pred_labels)))
print "Accuracy : "
print pred_true.count(True)/len(pred_true)

# sample data predict
for comment in xx['comment_text']:
    p = np.atleast_1d(np.array([comment,]))
    predicted = classifier.predict(p)
    print predicted
    all_labels = mlb.inverse_transform(predicted)
    print all_labels
    #print classifier.predict_proba(p)
'''

#snips data
'''
predicted = classifier.predict(X_test)
#print predicted
#print classifier.predict_proba(X_test)
all_labels = mlb.inverse_transform(predicted)
pred_true = []
for item, org_labels, pred_labels in zip(X_test, y_org, all_labels):
    #print item,"***", org_labels,"###", pred_labels
    pred_true.append(all(True if i in org_labels else False for i in pred_labels))
    #print('{0} => {1} => {2}'.format(item, ', '.join(org_labels), ', '.join(pred_labels)))
print "Accuracy : "
print pred_true.count(True)/len(pred_true)

# snips sample data testing for multi-intent
sample1 = 'Book a reservation for an oyster bar. with that add the album to my  Club Hits playlist.'
sample2 = 'book The Middle East  restaurant in IN for noon and I want to add another album to the wine & dine playlist'
p = np.atleast_1d(np.array([sample1,]))
predicted = classifier.predict(p)
#print predicted
#print classifier.predict_proba(X_test)
all_labels = mlb.inverse_transform(predicted)
print "sample class : ",all_labels
'''

# ice-xd :: testAPSSDC sample data testing for multi-intent
sample1 = 'hi, May I know the job fair detail of asasasas and  job openings in dsdsdsdsdsdsd'
sample2 = 'Give the grrtrtrt course application status and give me yuy course assessment details'
sample3 = 'OK Thanks. can i get your job role specification JOBROLE'
sample4 = 'May I know details of job PERSON details any courses available in SECTOR sector ?'
#sample_y = [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0]
#for index in [itemindex1[0][0],itemindex2[0][0]]:
#    sample_y[index] = 1
sample1_y = [0 ,0 ,0 ,1 ,0 ,0 ,1 ,0 ,0 ,0]
sample2_y = [0 ,0 ,0 ,0 ,1 ,0 ,0 ,1 ,0 ,0]

print "intents list :: {0} \n".format(mlb.classes_)

p = np.atleast_1d(np.array([sample1,]))
predicted = classifier.predict(p)
print "predict intents index :: {0} \n".format(predicted[0])


all_labels = mlb.inverse_transform(predicted)
print "predicted intents :: {} ".format(all_labels)

predict_score = classifier.predict_proba(p)
print "predict intents prob :: {0} \n".format(predict_score)

#'''
# get 1,2,3 max when CalibratedClassifierCV or logisticReg used.
flat=predict_score.flatten()
flat.sort()
itemindex1 = np.where(predict_score[0]==flat[-1])
print "first intent '{0}' with prob {1} \n".format(mlb.classes_[itemindex1[0][0]],flat[-1])
itemindex2 = np.where(predict_score[0]==flat[-2])
print "second intent '{0}' with prob {1} \n".format(mlb.classes_[itemindex2[0][0]],flat[-2])
itemindex3 = np.where(predict_score[0]==flat[-3])
print "third intent '{0}' with prob {1} \n".format(mlb.classes_[itemindex3[0][0]],flat[-3])
#'''

# test score
#print "score :: {}".format(classifier.score(p,np.array([sample1_y])))

all_labels = mlb.inverse_transform(predicted)
print "predicted intents :: {} ".format(all_labels)


intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 



[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.

predict intents index :: [0 0 0 1 0 0 1 0 0 0] 

predicted intents :: [(u'get_Jobfair_details', u'get_job_details')] 


[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elaps

predict intents prob :: [[0.    0.004 0.    0.572 0.026 0.    0.534 0.01  0.    0.   ]] 

first intent 'get_Jobfair_details' with prob 0.572 

second intent 'get_job_details' with prob 0.534 

third intent 'get_assessment_details' with prob 0.026 

predicted intents :: [(u'get_Jobfair_details', u'get_job_details')] 


[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished


# prediction OneVsOne

In [13]:
# custom data
'''
predicted = classifier.predict(X_test)
print predicted
all_labels = mlb.inverse_transform(predicted)

for item, labels in zip(X_test, all_labels):
    print('{0} => {1}'.format(item, ', '.join(labels)))
'''

# toxic data
'''
test_label_toxic = [tuple(j for j in i if j!="NA") for i in test_labels_df[cols_target].values.tolist()]
pred_true = []
for item, org_labels, pred_labels in zip(X_test, test_label_toxic, all_labels):
    pred_true.append(all(True if i in org_labels else False for i in pred_labels))
    #print('{0} => {1} => {2}'.format(item, ', '.join(org_labels), ', '.join(pred_labels)))
print "Accuracy : "
print pred_true.count(True)/len(pred_true)

# sample data predict
for comment in xx['comment_text']:
    p = np.atleast_1d(np.array([comment,]))
    predicted = classifier.predict(p)
    print predicted
    all_labels = mlb.inverse_transform(predicted)
    print all_labels
'''

#snips data
'''
predicted = classifier.predict(X_test)
#print predicted
all_labels = mlb.inverse_transform(predicted)
pred_true = []
for item, org_labels, pred_labels in zip(X_test, y_org, all_labels):
    #print item,"***", org_labels,"###", pred_labels
    pred_true.append(all(True if i in org_labels else False for i in pred_labels))
    #print('{0} => {1} => {2}'.format(item, ', '.join(org_labels), ', '.join(pred_labels)))
print "Accuracy : "
print pred_true.count(True)/len(pred_true)

# snips sample data testing for multi-intent
sample1 = 'Book a reservation for an oyster bar. with that add the album to my  Club Hits playlist.'
sample2 = 'book The Middle East  restaurant in IN for noon and I want to add another album to the wine & dine playlist'
p = np.atleast_1d(np.array([sample1,]))
predicted = classifier.predict(p)
#print predicted
all_labels = mlb.inverse_transform(predicted)
print "sample class : ",all_labels
'''

# ice-xd :: testAPSSDC sample data testing for multi-intent
sample1 = 'hi, May I know the job fair detail of asasasas and  job openings in dsdsdsdsdsdsd'
sample2 = 'Give the grrtrtrt course application status and give me yuy course assessment details'
sample3 = 'OK Thanks. can i get your job role specification JOBROLE'
sample4 = 'May I know details of job PERSON details any courses available in SECTOR sector ?'
#sample_y = [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0]
#for index in [itemindex1[0][0],itemindex2[0][0]]:
#    sample_y[index] = 1
sample1_y = [0 ,0 ,0 ,1 ,0 ,0 ,1 ,0 ,0 ,0]
sample2_y = [0 ,0 ,0 ,0 ,1 ,0 ,0 ,1 ,0 ,0]

print "intents list :: {0} \n".format(mlb.classes_)

p = np.atleast_1d(np.array([sample1,]))
predicted = classifier.predict(p)
print "predict intents index :: {0} \n".format(predicted[0])

#print help(classifier.predict_proba)

all_labels = mlb.inverse_transform(predicted)
print "predicted intents :: {} ".format(all_labels)

try:
    predict_score = classifier.predict_proba(p)
    print "predict intents prob :: {0} \n".format(predict_score)
    print "** ",predict_score[:,1]
    #print np.round(predict_score,2)

    #'''
    # get 1,2,3 max when CalibratedClassifierCV or logisticReg used.
    flat=predict_score.flatten()
    flat.sort()
    itemindex1 = np.where(predict_score[0]==flat[-1])
    print "first intent '{0}' with prob {1} \n".format(mlb.classes_[itemindex1[0][0]],flat[-1])
    itemindex2 = np.where(predict_score[0]==flat[-2])
    print "second intent '{0}' with prob {1} \n".format(mlb.classes_[itemindex2[0][0]],flat[-2])
    itemindex3 = np.where(predict_score[0]==flat[-3])
    print "third intent '{0}' with prob {1} \n".format(mlb.classes_[itemindex3[0][0]],flat[-3])
    #'''
except:
    pass

all_labels = mlb.inverse_transform(predicted)
print "predicted intents :: {} ".format(all_labels)


intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 



[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished


predict intents index :: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

predicted intents :: [()] 
predict intents prob :: [array([[1., 0.]]), array([[0.994, 0.006]]), array([[1., 0.]]), array([[0.534, 0.466]]), array([[0.988, 0.012]]), array([[1., 0.]]), array([[0.536, 0.464]]), array([[0.948, 0.052]]), array([[1., 0.]]), array([[1., 0.]])] 

**  predicted intents :: [()] 


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished


# good model . but memory consumption high.

In [4]:

from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

classifier_new = MLkNN(k=10)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(X_train)
vectorizer.fit(X_test)

x_train = vectorizer.transform(X_train)

#y_train = vectorizer.transform(y_train_text)
x_test = vectorizer.transform(X_test)

# Note that this classifier can throw up errors when handling sparse matrices.

x_train = lil_matrix(x_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(x_test).toarray()

# train
classifier_new.fit(x_train, y_train)

# predict
predictions_new = classifier_new.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_new))
print("\n")

MemoryError: 

# Estimation

# SGDClassifier with gridsearchCV
# sample1
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 0 0 0 1 0 0 0] 

predict intents prob :: [[0.14845367 0.00999279 0.0081006  0.4624159  0.02503919 0.00444298
  0.52334891 0.00770991 0.00615307 0.00859134]] 

first intent 'get_job_details' with prob 0.523348906737 

second intent 'get_Jobfair_details' with prob 0.462415899745 

third intent 'Greetings' with prob 0.148453672872 

predicted intents :: [(u'get_job_details',)] 
        
# sample2
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 0 1 0 0 0 0 0] 

predict intents prob :: [[0.01248607 0.01102929 0.00662873 0.10694141 0.84651563 0.00699763
  0.00548128 0.19659661 0.02024358 0.00762391]] 

first intent 'get_assessment_details' with prob 0.846515625599 

second intent 'getcoursedetails' with prob 0.196596611036 

third intent 'get_Jobfair_details' with prob 0.10694141261 

predicted intents :: [(u'get_assessment_details',)] 

# SGD without gridsearchCV
# sample1
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 0 0 0 0 0 0 0] 

predict intents prob :: [[4.71098626e-02 2.03569200e-02 3.01329079e-03 2.96368792e-01
  3.33655618e-04 2.03372786e-03 2.51610079e-01 1.00191516e-04
  3.11721327e-03 2.25364239e-03]] 

first intent 'get_Jobfair_details' with prob 0.296368791512 

second intent 'get_job_details' with prob 0.251610078825 

third intent 'Greetings' with prob 0.0471098625678 

predicted intents :: [()] 
# sample2
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 0 1 0 0 0 0 0] 

predict intents prob :: [[0.00230899 0.01561245 0.00656202 0.01591573 0.99986673 0.00464843
  0.01037208 0.13693212 0.0149582  0.00198682]] 

first intent 'get_assessment_details' with prob 0.999866729937 

second intent 'getcoursedetails' with prob 0.136932118973 

third intent 'get_Jobfair_details' with prob 0.0159157289639 

predicted intents :: [(u'get_assessment_details',)] 

# svm.SVC with gridsearchCV
# sample1
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 0 0 0 0 0 0 0] 

predict intents prob :: [[1.68392978e-01 2.82490942e-02 8.51079907e-03 4.96766230e-01
  1.85874001e-03 6.52101475e-03 4.61781836e-01 3.95648082e-04
  3.98004865e-03 7.27558009e-03]] 

first intent 'get_Jobfair_details' with prob 0.496766229514 

second intent 'get_job_details' with prob 0.461781835736 

third intent 'Greetings' with prob 0.1683929782 

predicted intents :: [()] 
# sample2
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 0 1 0 0 0 0 0] 

predict intents prob :: [[0.00721767 0.02014245 0.0081212  0.01055531 0.99996965 0.0067766
  0.0129972  0.13290952 0.00840004 0.00700586]] 

first intent 'get_assessment_details' with prob 0.99996964881 

second intent 'getcoursedetails' with prob 0.132909515458 

third intent 'No intent' with prob 0.0201424480183 

predicted intents :: [(u'get_assessment_details',)] 

# svm.SVC without gridSearchCV
# sample1
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 1 0 0 1 0 0 0] 

predict intents prob :: [[0.39198073 0.03528569 0.00725001 0.62577404 0.00397502 0.00422488
  0.84447324 0.00186998 0.00605879 0.00674809]] 

first intent 'get_job_details' with prob 0.844473238473 

second intent 'get_Jobfair_details' with prob 0.625774041331 

third intent 'Greetings' with prob 0.391980733892 

predicted intents :: [(u'get_Jobfair_details', u'get_job_details')] 
# sample2
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 0 1 0 0 0 0 0] 

predict intents prob :: [[0.00916612 0.02612597 0.00819797 0.02594459 0.99999559 0.00584121
  0.01214022 0.06572587 0.0099974  0.00999234]] 

first intent 'get_assessment_details' with prob 0.999995586966 

second intent 'getcoursedetails' with prob 0.0657258670384 

third intent 'No intent' with prob 0.0261259650633 

predicted intents :: [(u'get_assessment_details',)] 

# LinearSVC
# sample1
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 1 0 0 0 0 0 0] 

predict intents prob :: [[2.63027463e-01 2.42802561e-02 7.65685202e-03 5.14094903e-01
  1.85059975e-03 5.92744812e-03 4.51604164e-01 3.40920662e-04
  7.19157501e-03 6.50554473e-03]] 

first intent 'get_Jobfair_details' with prob 0.514094902547 

second intent 'get_job_details' with prob 0.451604164483 

third intent 'Greetings' with prob 0.26302746298 

predicted intents :: [(u'get_Jobfair_details',)] 
# sample2
intents list :: [u'Greetings' u'No intent' u'conclusionflow' u'get_Jobfair_details'
 u'get_assessment_details' u'get_document_details' u'get_job_details'
 u'getcoursedetails' u'jobfair_reg_details' u'successflow'] 

predict intents index :: [0 0 0 0 1 0 0 0 0 0] 

predict intents prob :: [[0.00625314 0.01737233 0.00754981 0.01038664 0.99997775 0.00631261
  0.00995603 0.1164047  0.00791785 0.00659245]] 

first intent 'get_assessment_details' with prob 0.999977750678 

second intent 'getcoursedetails' with prob 0.116404696589 

third intent 'No intent' with prob 0.0173723341191 

predicted intents :: [(u'get_assessment_details',)] 